## Import Libraries

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%%capture
!pip install yfinance
!pip install pyts
!pip installpandas_market_calendars
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
import logging
import warnings
import pyts
import pickle

warnings.filterwarnings('ignore', message='The frame.append method is deprecated')
yfinance_logger = logging.getLogger('yfinance')
yfinance_logger.setLevel(logging.CRITICAL)
yfinance_logger.propagate = False

## Generate Time Series (RAW)

This code segment processes a CSV file containing historical data on S&P500 constituents from a public [GitHub repository](https://github.com/fja05680/sp500) by user fja05680 <br> to generate a cleaned DataFrame with a complete date range and filled missing ticker values.

In [ ]:
# Access the CSV file containing historical data on S&P500 constituents
data_url = 'https://raw.githubusercontent.com/sukilovm/Encoding-financial-time-series-as-images/main/S%26P500_Historical_Constituents.csv'
raw_data = pd.read_csv(data_url)
sp500_hist_constituents = raw_data

# Convert the 'date' column to datetime objects
sp500_hist_constituents['date'] = pd.to_datetime(sp500_hist_constituents['date'])

# Set 'date' as the index
sp500_hist_constituents.set_index('date', inplace=True)

# Create a new DataFrame with a DatetimeIndex containing all dates between the minimum and maximum dates in 'date'
all_dates = pd.date_range(sp500_hist_constituents.index.min(), sp500_hist_constituents.index.max())
index_df = pd.DataFrame(index=all_dates)

# Join the original DataFrame with the new DataFrame
combined_df = index_df.join(sp500_hist_constituents)

# Use the ffill() method to fill the missing values with the previous non-null value
combined_df['tickers'].ffill(inplace=True)

# Reset the index to have 'date' as a regular column
combined_df.reset_index(inplace=True)
combined_df.rename(columns={'index': 'date'}, inplace=True)

combined_df

,date,tickers
0,1996-01-02,"AAL,AAMRQ,AAPL,ABI,ABS,ABT,ABX,ACKH,ACV,ADM,AD..."
1,1996-01-03,"AAL,AAMRQ,AAPL,ABI,ABS,ABT,ABX,ACKH,ACV,ADM,AD..."
2,1996-01-04,"AAL,AAMRQ,AAPL,ABI,ABS,ABT,ABX,ACKH,ACV,ADM,AD..."
3,1996-01-05,"AAL,AAMRQ,AAPL,ABI,ABS,ABT,ABX,ACKH,ACV,ADM,AD..."
4,1996-01-06,"AAL,AAMRQ,AAPL,ABI,ABS,ABT,ABX,ACKH,ACV,ADM,AD..."
...,...,...
9936,2023-03-16,"A,AAL,AAP,AAPL,ABBV,ABC,ABT,ACGL,ACN,ADBE,ADI,..."
9937,2023-03-17,"A,AAL,AAP,AAPL,ABBV,ABC,ABT,ACGL,ACN,ADBE,ADI,..."
9938,2023-03-18,"A,AAL,AAP,AAPL,ABBV,ABC,ABT,ACGL,ACN,ADBE,ADI,..."
9939,2023-03-19,"A,AAL,AAP,AAPL,ABBV,ABC,ABT,ACGL,ACN,ADBE,ADI,..."


This code segment defines a function which results in a DataFrame containing the earliest and latest dates for each ticker in the original data.

Afterwards, these dates and ticker symbols are used in combination with the Yahoo! Finance library to retrieve the daily stock data for each stock within the defined date range.

In [ ]:
def get_date_ranges(df):
    ticker_dates = {}

    for index, row in df.iterrows():
        tickers = row['tickers'].split(',')
        date = row['date']

        for ticker in tickers:
            if ticker not in ticker_dates:
                ticker_dates[ticker] = {'earliest': date, 'latest': date}
            else:
                if date < ticker_dates[ticker]['earliest']:
                    ticker_dates[ticker]['earliest'] = date
                if date > ticker_dates[ticker]['latest']:
                    ticker_dates[ticker]['latest'] = date

    new_df = pd.DataFrame.from_dict(ticker_dates, orient='index').reset_index()
    new_df.columns = ['ticker', 'earliest_date', 'latest_date']

    return new_df

date_ranges_df = get_date_ranges(combined_df)
date_ranges_df

,ticker,earliest_date,latest_date
0,AAL,1996-01-02,2023-03-20
1,AAMRQ,1996-01-02,2003-03-13
2,AAPL,1996-01-02,2023-03-20
3,ABI,1996-01-02,2008-11-23
4,ABS,1996-01-02,2006-06-01
...,...,...,...
1137,STLD,2022-12-22,2023-03-20
1138,GEHC,2023-01-04,2023-03-20
1139,BG,2023-03-15,2023-03-20
1140,PODD,2023-03-15,2023-03-20


In [ ]:
# Replace earliest_date with cutoff date
cutoff_date = pd.to_datetime('2012-12-03')

# Replace the values in the 'earliest_date' column if they are less than the cutoff_date
date_ranges_df.loc[date_ranges_df['earliest_date'] < cutoff_date, 'earliest_date'] = cutoff_date

def get_stock_price(ticker, earliest_date, latest_date):
    try:
        start_date = datetime.datetime.strptime(earliest_date, '%Y-%m-%d')
        end_date = datetime.datetime.strptime(latest_date, '%Y-%m-%d')

        # Check if start_date is after end_date
        if start_date > end_date:
            return None

        ticker_obj = yf.Ticker(ticker)
        stock_data = ticker_obj.history(start=start_date, end=end_date)

        if stock_data.empty:
            return None
        else:
            return stock_data
    except Exception:
        return None

def get_stock_data_for_date_ranges(date_ranges_df):
    stock_data_list = []

    for index, row in date_ranges_df.iterrows():
        ticker = row['ticker']
        earliest_date = row['earliest_date'].strftime('%Y-%m-%d')
        latest_date = row['latest_date'].strftime('%Y-%m-%d')

        stock_data = get_stock_price(ticker, earliest_date, latest_date)

        if stock_data is not None:
            stock_data['ticker'] = ticker
            stock_data_list.append(stock_data)

    combined_stock_data = pd.concat(stock_data_list)
    return combined_stock_data

combined_stock_data = get_stock_data_for_date_ranges(date_ranges_df)
combined_stock_data

,Open,High,Low,Close,Volume,Dividends,Stock Splits,ticker,Capital Gains
Date,,,,,,,,,
2012-12-03 00:00:00-05:00,12.180219,12.180219,11.897397,11.935106,3167000,0.0,0.0,AAL,NaN
2012-12-04 00:00:00-05:00,11.991672,12.151938,11.369463,11.671140,5775300,0.0,0.0,AAL,NaN
2012-12-05 00:00:00-05:00,11.746558,11.803123,11.284615,11.416598,6484300,0.0,0.0,AAL,NaN
2012-12-06 00:00:00-05:00,11.435454,11.953962,11.407172,11.887970,7057000,0.0,0.0,AAL,NaN
2012-12-07 00:00:00-05:00,11.935107,12.302776,11.737131,12.038809,9250800,0.0,0.0,AAL,NaN
...,...,...,...,...,...,...,...,...,...
2023-03-16 00:00:00-04:00,94.437299,97.386604,94.437299,96.046013,2212300,0.0,0.0,BG,NaN
2023-03-17 00:00:00-04:00,95.678591,96.264484,91.329116,91.468140,3876800,0.0,0.0,BG,NaN
2023-03-15 00:00:00-04:00,309.600006,312.760010,302.049988,307.890015,1605600,0.0,0.0,PODD,NaN


In [ ]:
# Reset index and ensure the 'Date' column is a datetime object
combined_stock_data = combined_stock_data.reset_index()
combined_stock_data['Date'] = pd.to_datetime(combined_stock_data['Date'])

# Extract the year from the 'Date' column and create a new 'year' column
combined_stock_data['year'] = combined_stock_data['Date'].dt.year

# Remove time components
combined_stock_data['Date'] = pd.to_datetime(combined_stock_data['Date']).dt.date

# Select only the desired columns
desired_columns = ["Date", "Open", "High", "Low", "Close", "Volume", "ticker", "year"]
combined_stock_data = combined_stock_data[desired_columns]

# Rename the columns
combined_stock_data = combined_stock_data.rename(columns={"ticker": "Ticker", "year": "Year" })

combined_stock_data

,Date,Open,High,Low,Close,Volume,Ticker,Year
0,2012-12-03,12.180219,12.180219,11.897397,11.935106,3167000,AAL,2012
1,2012-12-04,11.991672,12.151938,11.369463,11.671140,5775300,AAL,2012
2,2012-12-05,11.746558,11.803123,11.284615,11.416598,6484300,AAL,2012
3,2012-12-06,11.435454,11.953962,11.407172,11.887970,7057000,AAL,2012
4,2012-12-07,11.935107,12.302776,11.737131,12.038809,9250800,AAL,2012
...,...,...,...,...,...,...,...,...
1156165,2023-03-16,94.437299,97.386604,94.437299,96.046013,2212300,BG,2023
1156166,2023-03-17,95.678591,96.264484,91.329116,91.468140,3876800,BG,2023
1156167,2023-03-15,309.600006,312.760010,302.049988,307.890015,1605600,PODD,2023
1156168,2023-03-16,306.000000,314.260010,304.750000,312.359985,899700,PODD,2023


This code segment fills missing data rows with the previous day's one. The missing rows result from trading holiday's in the US.

Afterwards, the processed data is cut into windows and labels, which later will be used for image generation.

In [ ]:
## Remove rows of tickers, with problematic data
problematic_tickers = ['BMC', 'COL', 'COV', 'HNZ', 'HOT', 'PCL']
combined_stock_data_2 = combined_stock_data.copy()
combined_stock_data_2 = combined_stock_data_2[~combined_stock_data_2['Ticker'].isin(problematic_tickers)]
combined_stock_data_2 = combined_stock_data_2.reset_index(drop=True)

## Fill missing trading holidays with previous available day's data

# Create a dataframe from the input data
df = combined_stock_data_2

# Convert the 'Date' column to pandas datetime
df['Date'] = pd.to_datetime(df['Date'])

# Create an empty dataframe to store the result
result = pd.DataFrame()

# Find the unique tickers in the dataset
unique_tickers = df['Ticker'].unique()

# Process each ticker separately
for ticker in unique_tickers:
    # Get the data for the current ticker
    ticker_data = df[df['Ticker'] == ticker].sort_values('Date')

    # Find the min and max date for the current ticker
    min_date = ticker_data['Date'].min()
    max_date = ticker_data['Date'].max()

    # Create a date range between the min and max date, including only weekdays
    date_range = pd.date_range(start=min_date, end=max_date, freq='B')  # 'B' is for business days (Monday-Friday)

    # Merge the ticker_data with the date_range, forward filling the missing data
    ticker_data = ticker_data.set_index('Date').reindex(date_range, method='ffill').reset_index().rename(columns={'index': 'Date'})
    ticker_data['Ticker'] = ticker

    # Append the processed ticker data to the result dataframe
    result = result.append(ticker_data, ignore_index=True)

# Print the result dataframe
result

,Date,Open,High,Low,Close,Volume,Ticker,Year
0,2012-12-03,12.180219,12.180219,11.897397,11.935106,3167000,AAL,2012
1,2012-12-04,11.991672,12.151938,11.369463,11.671140,5775300,AAL,2012
2,2012-12-05,11.746558,11.803123,11.284615,11.416598,6484300,AAL,2012
3,2012-12-06,11.435454,11.953962,11.407172,11.887970,7057000,AAL,2012
4,2012-12-07,11.935107,12.302776,11.737131,12.038809,9250800,AAL,2012
...,...,...,...,...,...,...,...,...
1195737,2023-03-16,94.437299,97.386604,94.437299,96.046013,2212300,BG,2023
1195738,2023-03-17,95.678591,96.264484,91.329116,91.468140,3876800,BG,2023
1195739,2023-03-15,309.600006,312.760010,302.049988,307.890015,1605600,PODD,2023
1195740,2023-03-16,306.000000,314.260010,304.750000,312.359985,899700,PODD,2023


In [ ]:
show_result = result[['Date','Close', 'Open', 'High', 'Low', 'Ticker', 'Year']]
show_result[:25]

,Date,Close,Open,High,Low,Ticker,Year
0,2012-12-03,11.935106,12.180219,12.180219,11.897397,AAL,2012
1,2012-12-04,11.671140,11.991672,12.151938,11.369463,AAL,2012
2,2012-12-05,11.416598,11.746558,11.803123,11.284615,AAL,2012
3,2012-12-06,11.887970,11.435454,11.953962,11.407172,AAL,2012
4,2012-12-07,12.038809,11.935107,12.302776,11.737131,AAL,2012
5,2012-12-10,12.048235,12.057663,12.302776,11.878542,AAL,2012
6,2012-12-11,12.142510,12.123655,12.500752,12.019954,AAL,2012
7,2012-12-12,11.972817,12.151939,12.331059,11.925680,AAL,2012
8,2012-12-13,12.227357,12.029381,12.321631,11.944534,AAL,2012
9,2012-12-14,12.095374,12.208503,12.397051,12.085946,AAL,2012


In [ ]:
df = result

def create_windows_and_labels(df, window_size, threshold=0.5):
    df['Date'] = pd.to_datetime(df['Date'])
    df = df.sort_values(by=['Ticker', 'Date'])

    windows = []
    labels = []
    max_years = []
    tickers = df['Ticker'].unique()

    for ticker in tickers:
        ticker_df = df[df['Ticker'] == ticker]

        for i in range(len(ticker_df) - window_size):
            window_start = ticker_df.iloc[i]['Date']

            # Check if the window starts on a Monday
            if window_start.weekday() == 0:
                window_end = ticker_df.iloc[i + window_size - 1]
                next_week_index = i + window_size + 4

                # Check if next_week_index is within the dataframe's length
                if next_week_index < len(ticker_df):
                    next_week_close = ticker_df.iloc[next_week_index]['Close']
                    window = ticker_df.iloc[i:i + window_size][['Date', 'Close', 'Ticker', 'Year', 'Open', 'High', 'Low']]
                    label = 1 if next_week_close > window_end['Close'] else 0

                    # Check if all the 'Close' values in the window are the same
                    if window['Close'].nunique() > 1:
                        # Calculate the percentage change between consecutive 'Close' values
                        window['Close_pct_change'] = window['Close'].pct_change()

                        # Check if there is any extreme percentage change (above the threshold)
                        extreme_change = any(window['Close_pct_change'].abs() > threshold)

                        if not extreme_change:
                            window = window.drop(columns=['Close_pct_change'])  # Drop 'Close_pct_change' column
                            windows.append(window)
                            labels.append(label)
                            max_years.append(window['Year'].max())

    return windows, labels, max_years

window_size = 20

# windows, labels, max_years = create_windows_and_labels(df, window_size)

The resulting time series windows are referred to as RAW in the thesis paper.



In [ ]:
## Upload windows and labels to Google Drive

# Define the file path
# file_path = "/content/drive/MyDrive/20230424_windows_and_labels.pkl"

# with open(file_path, 'wb') as f:
#    pickle.dump((windows, labels, max_years), f)